In [2]:
import requests
from pyquery import PyQuery as pq
import pandas as pd
import re
import tqdm
import dask.bag as db
from dask.distributed import Client
import json
import numpy as np

In [3]:
BASE_URL = "https://results.chicagomarathon.com/2021/"
PATH = "?page={page}&event=MAR&lang=EN_CAP&num_results=1000&pid=list&search%5Bsex%5D={sex}&search%5Bage_class%5D=%25"

In [4]:
# 전역 변수로 예외 횟수 기록용 딕셔너리 생성
split_fail_count = {}

def safe_split(text, index=1, default="", field_name="unknown"):
    try:
        return text.split("\n")[index]
    except (IndexError, AttributeError):
        # 실패한 필드명을 기준으로 카운트 증가
        split_fail_count[field_name] = split_fail_count.get(field_name, 0) + 1
        return default
        
def parse_page(base_url, path, gender):
    resp = requests.get(base_url+path)
    d = pq(resp.content)
    # find first name field and navigate up to overarching row
    all_runners = d(".list-field.type-fullname a").closest(".list-group-item .row")
    all_runners_parsed = []
    for runner in all_runners.items():
    #     print(runner)
        name_country = runner.find(".type-fullname a").text()
        idp = re.search("(?<=idp=)[A-Z0-9_.-]*(?=&)", runner.find(".type-fullname a").attr['href']).group(0)
        details_url = base_url + "?content=detail&idp=" + idp
        # data = {
        #     "name": name_country[:-6],
        #     "gender": gender,
        #     "country": name_country[-4:-1],
        #     "age_class": runner.find(".type-age_class").text().split("\n")[1],
        #     "half_time": runner.find(".type-time").eq(0).text().split("\n")[1],
        #     "finish_time": runner.find(".type-time").eq(1).text().split("\n")[1],
        #     "details_url": details_url,
        # }
        data = {
            "name": name_country[:-6],
            "gender": gender,
            "country": name_country[-4:-1],
            "age_class": safe_split(runner.find(".type-age_class").text(), 1),
            "half_time": safe_split(runner.find(".type-time").eq(0).text(), 1),
            "finish_time": safe_split(runner.find(".type-time").eq(1).text(), 1),
            "details_url": details_url,
        }        
        all_runners_parsed.append(data)

    return all_runners_parsed

def get_details(details_url):
    x = pq(details_url)
    splits = {
        "start": {
            "time_of_day": x.find(".f-starttime_net.last").text(),
            "time": "00:00:00"
        },
        "5km": {
            "time_of_day": x.find(".f-time_05 .time_day").text(),
            "time": x.find(".f-time_05 .time").text()
        },
        "10km": {
            "time_of_day": x.find(".f-time_10 .time_day").text(),
            "time": x.find(".f-time_10 .time").text()
        },
        "15km": {
            "time_of_day": x.find(".f-time_15 .time_day").text(),
            "time": x.find(".f-time_15 .time").text()
        },
        "20km": {
            "time_of_day": x.find(".f-time_20 .time_day").text(),
            "time": x.find(".f-time_20 .time").text()
        },
        "half": {
            "time_of_day": x.find(".f-time_52 .time_day").text(),
            "time": x.find(".f-time_52 .time").text()
        },
        "25km": {
            "time_of_day": x.find(".f-time_25 .time_day").text(),
            "time": x.find(".f-time_25 .time").text()
        },
        "30km": {
            "time_of_day": x.find(".f-time_30 .time_day").text(),
            "time": x.find(".f-time_30 .time").text()
        },
        "35km": {
            "time_of_day": x.find(".f-time_35 .time_day").text(),
            "time": x.find(".f-time_35 .time").text()
        },
        "40km": {
            "time_of_day": x.find(".f-time_40 .time_day").text(),
            "time": x.find(".f-time_40 .time").text()
        },
        "finish": {
            "time_of_day": x.find(".f-time_finish_netto .time_day").text(),
            "time": x.find(".f-time_finish_netto .time").text()
        }
    }
    
    return {
        "bib": x.find(".f-start_no_text.last").text(),
        "city_state": x.find(".f-__city_state.last").text(),
        "splits": splits,
    }

In [5]:
all_runners = []

for page in tqdm.tqdm(range(1, 16)):
    all_runners += parse_page(BASE_URL, PATH.format(page=page, sex="M"), gender="man")
    
    
for page in tqdm.tqdm(range(1, 13)):
    all_runners += parse_page(BASE_URL, PATH.format(page=page, sex="W"), gender="woman")

  7%|▋         | 1/15 [00:10<02:29, 10.66s/it]


KeyboardInterrupt: 

In [ ]:
pd.DataFrame(all_runners).to_csv("data/runners.csv", index=None)

NameError: name 'pd' is not defined

In [ ]:
all_runners = pd.read_csv("data/runners.csv").to_dict(orient="records")

In [ ]:
client = Client()  # set up local cluster on your laptop
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50755,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.73 GiB
Comm: tcp://127.0.0.1:50780,Total threads: 2
Dashboard: http://127.0.0.1:50785/status,Memory: 1.93 GiB
Nanny: tcp://127.0.0.1:50758,


In [ ]:
len(db.from_sequence(all_runners).map(lambda x: x["name"]).compute())

27000

In [ ]:
all_runners[0]

{'name': 'Kiptum, Kelvin',
 'gender': 'man',
 'country': 'KEN',
 'age_class': '20-24',
 'half_time': '02:00:35',
 'finish_time': nan,
 'details_url': 'https://results.chicagomarathon.com/2021/?content=detail&idp=9TGG963828AFFB'}

In [ ]:
def add_details(runner):
    runner = runner.copy()
    try:
        details = get_details(runner["details_url"])
        runner.update(details)
        return runner
    except:
        print(runner)
        return {}
    

all_with_details = db.from_sequence(all_runners, npartitions=10000).map(add_details).compute()

In [ ]:
with open('data/runners_with_data.json', 'w') as fp:
    json.dump(all_with_details, fp)

In [ ]:
df

In [ ]:
df = pd.DataFrame(pd.json_normalize(all_with_details))

to_drop = ["splits.start.time", "details_url", "city_state", "finish_time", "half_time", "name", "country", "bib", "gender", "age_class"] 
# drop time of day for now
to_drop += [c for c in df.columns if "time_of_day" in c]
df = df.drop(columns=to_drop)

# handle missing value marker
df = df.replace("–", np.nan)

df.columns = [c.replace("splits.", "") for c in df.columns]
df.to_csv("./chicago_data.csv", index=None)